<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>

# __Embeddings__

Estimated time needed: **35** minutes


You work for a streaming service that wants an automatic way to determine if they should add a movie to its catalog; each film is expensive to purchase, so the  streaming service would only like good movies to add to the catalog. You have been hired to build a neural network to determine if a movie is good or not using critics' written ratings similar to "Rotten Tomatoes". 

In this notebook, you will learn the fundamentals of how to convert text data to usable features for your neural network. In addition, you will also learn about the embedding layer, an engineering solution that allows you to implement linear layers for categorical data effectively. 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/movie_review.webp" style="width: 60%">


## Objectives

After completing this lab, you will be able to:

 - Understand the theory behind Embedding Layers  
 - Apply Tokenizer, Embedding Layers to a neural network 
 - Perform Sentiment Analysis using a neural network 


----


## Setup

For this lab, we will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run these notebook commands in a different Jupyter environment (like Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the corresponding code cell below.


In [2]:
%%capture
!pip install --upgrade tensorflow

### Importing Required Libraries


In [12]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import importlib
import numpy as np

import matplotlib.pyplot as plt
from numpy.random import seed
seed(1)

import  tensorflow as tf

tf.random.set_seed(1234)
from tensorflow.keras.layers import Embedding, Dense, Flatten,Dropout
from tensorflow.keras.models import Sequential

from keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import regularizers


ImportError: Traceback (most recent call last):
  File "C:\Users\samet\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Devingen bağlantı kitaplığını (DLL) başlatma işlemi başarısız.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [7]:
# Print tensorflow version, should be greater than 2.9.0
print(tf.__version__)

NameError: name 'tf' is not defined

### Defining Helper Functions


This function will plot the neural network's loss and accuracy curves from its training history:


In [ ]:
def display_metrics(history):

    n = len(history.history["loss"])

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(range(n), (history.history["loss"]),'r', label="Train Loss")
    ax.plot(range(n), (history.history["val_loss"]),'b', label="Validation Loss")
    ax.legend()
    ax.set_title('Loss over iterations')

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(range(n), (history.history["acc"]),'r', label="Train Acc")
    ax.plot(range(n), (history.history["val_acc"]),'b', label="Validation Acc")
    ax.legend(loc='lower right')
    ax.set_title('Accuracy over iterations')

This function will plot word embeddings that have been reduced to two dimensions:


In [ ]:
def plot_embedding(X_embedded,start=100,stop=300,sample=10):
    fig, ax = plt.subplots()
    ax.scatter(X_embedded[start:stop:sample,0], X_embedded[start:stop:sample,1])
    for i in range(start,stop,sample):
        ax.annotate(REVERSE_LOOKUP[i+1], (X_embedded[i,0], X_embedded[i,1]))

This function will convert a probability to a Rotten Tomatoes score:


In [ ]:
def rotten_tomato_score(p_yx):
    return ["rotten"  if p<=0.60 else "fresh"for p in p_yx ]
        

##  Background


As most machine learning models need their input variables to be numeric, text variables need to be transformed in the pre-processing part. There are several methods to perform this transformation. One-hot and Multi-hot encoding are frequently used to deal with textual data. 

One-hot encoding involves creating a set of vectors whose length equals the number of unique elements in the corpus or the vocabulary. We represent individual words as ones; else, the element is zero. Consider the following list of three **sequences**:


In [ ]:
samples=['I hate cats', 
         'the dog is brown and I like cats', 
         'for the']

We have three **sequences**, and the corresponding number of **word tokens** for each sequence is given by:


1. I hate cats - 3 tokens 

2. the dog is brown and likes cats - 8 tokens

3. for the - 2 tokens




### Tokenizer


One way to encode text features to numeric features is using <b>one-hot-encoding</b> or <b>multi-hot-encoding</b>. The one-hot and multi-hot-encoding vector is a vector consisting of all zeros; if the sequence of text contains a specific word, then an element of the vector is set to one. Consider the vocabulary that consists of all the words in the list ```samples``` each vector will have an element assigned to it shown here:


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/one_hot.png" width="200" height="250">


Keras has an additional component for words not in the vocabulary that we will leave out in this explanation. We have ten words; therefore, the vector will have ten elements. For example, we have the vector for the word ```I``` given by:


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/I.png" width="550" height="650">


All the elements in the vector are zero except for the index corresponding to the word ```I```; this is an example of one-hot-encoding. For **multi-hot-encoding** the sentence 'I hate cats', the vector is given by the following:


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/I_hate_cats.png" width="550" height="650">


We can also represent the sequence as three one-hot-encoding vectors. We see that for each word that occurs, the value in the vector is set to one; else, they are set to zero; the dimension of both encoding vectors is equal to the number of unique words in the corpus (vocabulary). 

Lets see how to convert a sequence of multi-hot-encoding vectors.


###  Embedding

An embedding layer in Keras can be used when we want to create the embedding. These embeddings are learnable parameters that embed high dimensional data into lower dimensional vectors. Lets assume we have the embedding for the vocabulary above, we can represent the vocabulary as a list of parameter vectors $[w_{I}, w_{cats}, w_{the}, w_{hate}, w_{dog}, w_{is}, w_{brown},w_{and}, w_{like }, w_{for} ] $ .   

Thus, we can obtain the embedding for the sentence **"I hate cats"** by first performing one-hot-encoding for each token (as shown on the right) and then performing the matrix multiplication with the parameter vectors on the left: 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/Ihat_cats_embeding.png" width="1000" height="1200">


We represent the matrix as a set of three **one-hot-encoded** column vectors. We see the resultant vector is 1x3 **(output dimension x input length)** as Keras transposes this result. The input length is the number of words in a sentence. In this case, the output dimension is one, but it is usually a free parameter that you set as the number of dimensions of the low-dimensional space. 

We see that the operation is wasteful as we are multiplying many vectors by zeros. In real world applications, the one-hot-encoding is thousands of dimensions long. As a result, we use an **embedding layer**; this layer simply **outputs the parameters given the indexes**, as shown below. We can use these embeddings as the input to a neural network. Note that the parameters of an embedding layer are randomly initialized and are updated when training the neural network, just like the fully connected layers:


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/I_hate_cats_embeding.png" width="1000" height="1200">


## Tokenizer in Keras


Keras has the class ``Tokenizer`` that allows you to vectorize a text by turning each word into either a sequence of integers or a vector where the coefficient for each token is binary, based on word count. We will use the following parameters:

```num_words```: the maximum number of words to keep in a sentence, based on the highest word frequencies, that is, only the **most common** ``num_words`` words will be kept.

```filters```: a string that contains characters that will be filtered from the texts. The default is all the punctuations, tabs, line breaks, and the ' character.

```lower```: boolean type, indicates whether to convert the texts to lowercase.

```split```: str, indicates the separator for word splitting.

We create the ```Tokenizer``` object:


In [ ]:
tokenizer = Tokenizer(num_words=11)

To fit the tokenizer to our text data, we call the method ```fit_on_texts```. The layer will build a corpus (vocabulary) of all string tokens obtained from the samples, and each word in the vocabulary will be represented by an integer or a vector.


In [ ]:
tokenizer.fit_on_texts(samples) 

The attribute ```word_counts``` is a list of words, tokens, and the corresponding number of times they appeared in the samples:


In [ ]:
word_counts=tokenizer.word_counts
word_counts

We can use the method ```texts_to_matrix``` to perform multi-hot encoding, where each element of the resultant vector is set to one if the corresponding word exists in the sequence; else, the element is set to zero. Here we perform multi-hot encoding for each word in the different sequences:


In [ ]:
for key in tokenizer.word_counts.keys():
    
    print(key)
    print(tokenizer.texts_to_matrix([key]))

As there is a total of ten words in the sequence, the vector has ten elements; each element in the vector corresponds to a different word. 


In [ ]:
for sample  in samples:
    
    print(sample)
    print(tokenizer.texts_to_matrix([sample]))

We can also apply different NLP transformations using the parameter ```mode```.


In [ ]:
modes=[ "binary", "count", "tfidf", "freq"]
for mode in modes: 
    print("mode:",mode)
    for sample  in samples:
        
        print(sample)
        print(tokenizer.texts_to_matrix([sample],mode=mode))

In many cases, Multi-hot encoding is redundant. As a result, we use the method ```texts_to_sequences```. This method will output an integer sequence where each integer corresponds to the index of the element in the corpus. The sequence length is equal to the number of words or tokens in the input sequence, as opposed to the number of words in the entire vocabulary like in Multi-hot encoding.


In [ ]:
for sample  in samples:
    
    print(sample)
    print(tokenizer.texts_to_sequences([sample]))





### Input Dimension 


Let's look at an example where we want to output the integer **input_dim**, which is the size of the vocabulary or maximum integer index.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input

import numpy as np


model = Sequential()

input_dim=3
output_dim=1
input_length=1
model.add(Embedding(input_dim=input_dim, output_dim=output_dim,input_length=input_length))

model.summary()

Keras randomly initialize the weights, so let's set each weight to the index it belongs to:


In [ ]:
model.get_weights()
weights=np.array([0,1,2]).reshape(-1,1)

model.set_weights([weights])
model.get_weights()

We can apply the function to each integer. The embedding layer will return the parameter value(s) or vector(s) on the position(s) which have the index value(s) equal to the integer(s) in the sequence.


In [ ]:
for n in range(3):
    x=np.array([[n]])
    print("input x={}".format(n))
    z=model.predict(x)
    print("output z={}".format(z.tolist()))

We can also apply the method to multiple samples, by making each sample a different sample in the batch dimension or as multiple samples in a row:


In [ ]:
z = model.predict([[0],[1],[2]])
print("different samples in the batch dimension:\n",z)
z = model.predict([0,1,2])
print(" multiple samples in a row: \n",z)

If you use the ```Sequential``` class, you must specify the input shape ```input_length``` so the model can be built with the proper set of parameters; ```input_length``` is the size of each input sequence.


### Output shape

Output shape is the number of parameters, using the matrix multiplication analogy: if the input is a multi-hot encoding column vector and the embedding is a matrix, the ```output_dim``` is the number of columns in the matrix. Consider the example of converting a number to its binary number equivalent, and we will represent each element of the binary number as a different dimension in the output. 

Let's convert the numbers from 0 to 3. As we have four numbers the parameter ```input_dim=4```. As we need two binary digits or bits to represent the numbers from one to three the ```output_dim=2```.


In [ ]:
input_dim=4
output_dim=2
input_length=1
model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
model.summary()

We assign the ```weights``` to represent binary numbers;


In [ ]:
weights=np.array([[0,0],[0,1],[1,0],[1,1]])
model.set_weights([weights])
model.get_weights()

For each input integer, the output of the embedding is the corresponding binary number:


In [ ]:
for n in range(4):
    x=np.array([[n]])
    print("input x={}".format(n))
    z=model.predict(x)
    print("input binary={}".format(z.tolist() ))

in order to use the embedding, we go with seqeuntial 


### Padding Sequences


Usually, sequences are of different lengths. Consider our original sequence:


In [ ]:
samples=['I hate cats','the dog is brown and I like cats','for the']
for sample in samples:
    print("sample:",sample)
    print("length:",len(sample))

When we apply the ```Tokenizer``` we apply the method ```texts_to_sequences``` we see each sequence is also a different length:


In [ ]:
tokenizer = Tokenizer(num_words=12)
tokenizer.fit_on_texts(samples) 
tokens=tokenizer.texts_to_sequences(samples)
print("tokens",tokens)

To make sure the sequences are the same length, we apply the function ```pad_sequences```. The function transforms a sequence into a 2D Numpy array with the number of rows equal to the number of samples and the number of columns equal to the parameter ```maxlen``` (the length of the longest sequence in the list). Sequences that are shorter than ```maxlen``` are padded with the parameter ```value```, which is by default zero. 

Consider the following example:


In [ ]:
maxlen=9
x =pad_sequences(tokens, maxlen=maxlen,value=0)
x

We see the padding proceeds the integer values, but we can also set padding to follow the integers through ```padding = "post"```.


In [ ]:
maxlen=9
x =pad_sequences(tokens, maxlen=maxlen,padding="post")
x

If we decrease the value for ```maxlen```, the function will ```truncate``` the sequences, that is, making sequences that are longer than `maxlen` shorter.


In [ ]:
maxlen=5
x =pad_sequences(tokens, maxlen=maxlen)
x

We see the function truncates the values from sequences longer than ```maxlen``` at the beginning of the sequence. We can also set ```truncating = "post"```to  truncate the values from the end.


## Sentiment Analysis 


Now that we have the sequence, , and embedding layers, we can use a neural network to classify our text data. Sentiment analysis in natural language processing is a technique used to determine opinions, appraisals, emotions, or attitudes towards a topic, person, or entity. 

Consider the IMDB dataset of 25,000 movie reviews from IMDB, labeled positive (y=0) and negative (y=1). In this dateset, the reviews have been pre-processed, and each review is encoded as a list of word indexes (integers). The words are indexed by overall frequency in the dataset so that for instance, the integer "3" encodes the 3rd most frequent word in the data. We will "only consider the top 10,000 most common words, but eliminate the top 20 most common words for more on the dataset check out <a href="https://keras.io/api/datasets/imdb/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01">here </a>. We download the dataset:


In [ ]:
from keras.datasets import imdb

In [ ]:
max_features = 10000

# change the default parameter of np to allow_pickle=True
np.load.__defaults__=(None, True, True, 'ASCII')
importlib.reload(np)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features )

We print out the first three samples and we see the dataset has been tokenized: 


In [ ]:
for i,x in enumerate(x_train[0:3]):
    print("Sequence:",i) 
    print(x)

We can obtain a dictionary look-up table with the word and the integer that represents it:


In [ ]:
word_index = imdb.get_word_index()

We invert the table so the key is the integer that represents the word, and the value is the word. We assign the new dictionary to the variable ```REVERSE_LOOKUP```:


In [ ]:
REVERSE_LOOKUP={value:key for key, value in word_index.items()}

The following function will use ```REVERSE_LOOKUP``` to convert an input sequence x to the original review. 


In [ ]:
def get_review(x):
     return' '.join([REVERSE_LOOKUP[index ] for index in x])

We can obtain the review for the first sample:


In [ ]:
get_review(x_train[0])

Similarly, we obtain the review for the second sample:


In [ ]:
get_review(x_train[1])

Each sample is of a different length; for example, we can print the length of the first three samples:


In [ ]:
for i,x in enumerate(x_train[0:3]):
    print("length {} of sample {}:".format(i,len(x)))


As a result, we need to pad all the sequences using ```pad_sequences```:


In [ ]:
maxlen=20
x_train =pad_sequences(x_train, maxlen=maxlen)
x_test =pad_sequences(x_test, maxlen=maxlen)


In [ ]:
x_test.shape

Now we can create the embedding layer. We know the number of words is ```max_features=10000``` and therefore ```input_dim=10000```. The dimension of the output is a parameter that we set. Let's set ```output_dim=8``` and ```input_length=1``` for now.

We also need to add a flattening layer using ```Flatten()```, which adds an extra dimension and will reshape the input as ```(batch, 1)```.


In [ ]:
model = Sequential()
model.add(Embedding(10000, 8, input_length=20))
model.add(Flatten())


We can now add a fully connected layer with `units=1` for classification:


In [ ]:
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()


Let's fit the model:


In [ ]:
history = model.fit(x_train, y_train, epochs=10,batch_size=30,validation_split=0.2)

We can plot the training history:


In [ ]:
display_metrics(history)

The output of the model is a probability as it ranges from 0 to 1, so we can use this as a proxy for the Rotten Tomatoes score:


In [ ]:
p_yx=model.predict(x_test[0:10])

We can obtain the the rating using the helper function ```rotten_tomato_score``` which maps a probability to a prediction:


In [ ]:
rotten_tomato_score(p_yx)

### Visualizing the embedding  weights


The weights in the embedding layer contain correlations between words. Let's obtain the embedding weights:


In [ ]:
weights=model.layers[0].get_weights()[0]

There are 10,000 words so there are 10,000 different parameter vectors, each has 8 dimensions:


In [ ]:
weights.shape

We can't visualize the vector as it's 8-dimensional, but we can use **T-distributed Stochastic Neighbor Embedding (TSNE)** to reduce the dimension to two for visualizing the embeddings on a 2D plot:


In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_embedded = tsne.fit_transform(weights)

Now that the embeddings are transformed let's use the helper function ```plot_embedding``` to visualize them. The starting word index is ```start```, and ```stop``` is the index of the last word; ```sample``` is the number of samples between samples.


In [ ]:
start=1
stop=600
sample=10
plot_embedding(X_embedded,start,stop,sample)


We see words like "war", "evil", and "killer" are clustered in the lower-right quadrant.


In [ ]:
start=1
stop=100
sample=1
plot_embedding(X_embedded,start,stop,sample)

### Exercise 1 

Recreate the model above but add a dense layer with 500 neurons (using relu activation), a dense layer with 250 neurons (using relu activation), and a final layer for classification, then use the function ```display_metrics``` to plot the results.


<details>
    <summary>Click here for Solution</summary>

```python
model = Sequential()
model.add(Embedding(10000, 8, input_length=20))
model.add(Flatten())

model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10,batch_size=30,validation_split=0.2)

display_metrics(history)
```
</details>


### Exercise 2 - Question: Does this model perform better then the last model? Why or why not?


<details>
    <summary>Click here for Solution</summary>


The second model suffers from over Overfitting

</details>


### Exercise 3 - Introduce L2 and dropout regulation to your model (there are many solutions, one is given here)


In [ ]:
# Enter your solution here



<details>
    <summary>Click here for Solution</summary>

```python
model = Sequential()
model.add(Embedding(max_features, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(500, kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.L2(l2=5e-3)))
model.add(Dropout(.4))

model.add(Dense(250, kernel_initializer='normal', activation='relu',kernel_regularizer=regularizers.L2(l2=5e-3)))
model.add(Dropout(.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) 
model.summary()

history = model.fit(x_train, y_train, epochs=10,batch_size=64, validation_split=0.2)
display_metrics(history)
```
</details>


## Congratulations! You have completed this lab!


## Author


[Joseph Santarcangelo](https://www.linkedin.com/in/david-pasternak-6b84a2208/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01)


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2022-03-28        | 0.1     | David Pasternak      | Created Lab    |
| 2022-05-10        | 0.2     | Sam Prokopchuk      | Complete Draft of Lab    |
| 2022-08-10        | 0.2     | Roxanne Li      | Review and edit Lab    |
| 2022-09-08        | 0.2     | Steve Hord      | QA pass and edits      |


## <h3 align="center"> © IBM Corporation 2022. All rights reserved. <h3/>
